In [11]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 11: Monkey in the Middle ---</h2><p>As you finally start making your way upriver, you realize your pack is much lighter than you remember. Just then, one of the items from your pack goes flying overhead. Monkeys are playing <a href="https://en.wikipedia.org/wiki/Keep_away" target="_blank">Keep Away</a> with your missing things!</p>
<p>To get your stuff back, you need to be able to predict where the monkeys will throw your items. After some careful observation, you realize the monkeys operate based on <em>how worried you are about each item</em>.</p>
<p>You take some notes (your puzzle input) on the items each monkey currently has, how worried you are about those items, and how the monkey makes decisions based on your worry level. For example:</p>
<pre><code>Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
Starting items: 54, 65, 75, 74
Operation: new = old + 6
Test: divisible by 19
If true: throw to monkey 2
If false: throw to monkey 0

Monkey 2:
Starting items: 79, 60, 97
Operation: new = old \* old
Test: divisible by 13
If true: throw to monkey 1
If false: throw to monkey 3

Monkey 3:
Starting items: 74
Operation: new = old + 3
Test: divisible by 17
If true: throw to monkey 0
If false: throw to monkey 1
</code></pre>

<p>Each monkey has several attributes:</p>
<ul>
<li><code>Starting items</code> lists your <em>worry level</em> for each item the monkey is currently holding in the order they will be inspected.</li>
<li><code>Operation</code> shows how your worry level changes as that monkey inspects an item. (An operation like <code>new = old * 5</code> means that your worry level after the monkey inspected the item is five times whatever your worry level was before inspection.)</li>
<li><code>Test</code> shows how the monkey uses your worry level to decide where to throw an item next.
  <ul>
  <li><code>If true</code> shows what happens with an item if the <code>Test</code> was true.</li>
  <li><code>If false</code> shows what happens with an item if the <code>Test</code> was false.</li>
  </ul>
</li>
</ul>
<p>After each monkey inspects an item but before it tests your worry level, your relief that the monkey's inspection didn't damage the item causes your worry level to be <em>divided by three</em> and rounded down to the nearest integer.</p>
<p>The monkeys take turns inspecting and throwing items. On a single monkey's <em>turn</em>, it inspects and throws all of the items it is holding one at a time and in the order listed. Monkey <code>0</code> goes first, then monkey <code>1</code>, and so on until each monkey has had one turn. The process of each monkey taking a single turn is called a <em>round</em>.</p>
<p>When a monkey throws an item to another monkey, the item goes on the <em>end</em> of the recipient monkey's list. A monkey that starts a round with no items could end up inspecting and throwing many items by the time its turn comes around. If a monkey is holding no items at the start of its turn, its turn ends.</p>
<p>In the above example, the first round proceeds as follows:</p>
<pre><code>Monkey 0:
  Monkey inspects an item with a worry level of 79.
    Worry level is multiplied by 19 to 1501.
    Monkey gets bored with item. Worry level is divided by 3 to 500.
    Current worry level is not divisible by 23.
    Item with worry level 500 is thrown to monkey 3.
  Monkey inspects an item with a worry level of 98.
    Worry level is multiplied by 19 to 1862.
    Monkey gets bored with item. Worry level is divided by 3 to 620.
    Current worry level is not divisible by 23.
    Item with worry level 620 is thrown to monkey 3.
Monkey 1:
  Monkey inspects an item with a worry level of 54.
    Worry level increases by 6 to 60.
    Monkey gets bored with item. Worry level is divided by 3 to 20.
    Current worry level is not divisible by 19.
    Item with worry level 20 is thrown to monkey 0.
  Monkey inspects an item with a worry level of 65.
    Worry level increases by 6 to 71.
    Monkey gets bored with item. Worry level is divided by 3 to 23.
    Current worry level is not divisible by 19.
    Item with worry level 23 is thrown to monkey 0.
  Monkey inspects an item with a worry level of 75.
    Worry level increases by 6 to 81.
    Monkey gets bored with item. Worry level is divided by 3 to 27.
    Current worry level is not divisible by 19.
    Item with worry level 27 is thrown to monkey 0.
  Monkey inspects an item with a worry level of 74.
    Worry level increases by 6 to 80.
    Monkey gets bored with item. Worry level is divided by 3 to 26.
    Current worry level is not divisible by 19.
    Item with worry level 26 is thrown to monkey 0.
Monkey 2:
  Monkey inspects an item with a worry level of 79.
    Worry level is multiplied by itself to 6241.
    Monkey gets bored with item. Worry level is divided by 3 to 2080.
    Current worry level is divisible by 13.
    Item with worry level 2080 is thrown to monkey 1.
  Monkey inspects an item with a worry level of 60.
    Worry level is multiplied by itself to 3600.
    Monkey gets bored with item. Worry level is divided by 3 to 1200.
    Current worry level is not divisible by 13.
    Item with worry level 1200 is thrown to monkey 3.
  Monkey inspects an item with a worry level of 97.
    Worry level is multiplied by itself to 9409.
    Monkey gets bored with item. Worry level is divided by 3 to 3136.
    Current worry level is not divisible by 13.
    Item with worry level 3136 is thrown to monkey 3.
Monkey 3:
  Monkey inspects an item with a worry level of 74.
    Worry level increases by 3 to 77.
    Monkey gets bored with item. Worry level is divided by 3 to 25.
    Current worry level is not divisible by 17.
    Item with worry level 25 is thrown to monkey 1.
  Monkey inspects an item with a worry level of 500.
    Worry level increases by 3 to 503.
    Monkey gets bored with item. Worry level is divided by 3 to 167.
    Current worry level is not divisible by 17.
    Item with worry level 167 is thrown to monkey 1.
  Monkey inspects an item with a worry level of 620.
    Worry level increases by 3 to 623.
    Monkey gets bored with item. Worry level is divided by 3 to 207.
    Current worry level is not divisible by 17.
    Item with worry level 207 is thrown to monkey 1.
  Monkey inspects an item with a worry level of 1200.
    Worry level increases by 3 to 1203.
    Monkey gets bored with item. Worry level is divided by 3 to 401.
    Current worry level is not divisible by 17.
    Item with worry level 401 is thrown to monkey 1.
  Monkey inspects an item with a worry level of 3136.
    Worry level increases by 3 to 3139.
    Monkey gets bored with item. Worry level is divided by 3 to 1046.
    Current worry level is not divisible by 17.
    Item with worry level 1046 is thrown to monkey 1.
</code></pre>
<p>After round 1, the monkeys are holding items with these worry levels:</p>
<pre><code>Monkey 0: 20, 23, 27, 26
Monkey 1: 2080, 25, 167, 207, 401, 1046
Monkey 2: 
Monkey 3: 
</code></pre>
<p>Monkeys 2 and 3 aren't holding any items at the end of the round; they both inspected items during the round and threw them all before the round ended.</p>
<p>This process continues for a few more rounds:</p>
<pre><code>After round 2, the monkeys are holding items with these worry levels:
Monkey 0: 695, 10, 71, 135, 350
Monkey 1: 43, 49, 58, 55, 362
Monkey 2: 
Monkey 3:

After round 3, the monkeys are holding items with these worry levels:
Monkey 0: 16, 18, 21, 20, 122
Monkey 1: 1468, 22, 150, 286, 739
Monkey 2:
Monkey 3:

After round 4, the monkeys are holding items with these worry levels:
Monkey 0: 491, 9, 52, 97, 248, 34
Monkey 1: 39, 45, 43, 258
Monkey 2:
Monkey 3:

After round 5, the monkeys are holding items with these worry levels:
Monkey 0: 15, 17, 16, 88, 1037
Monkey 1: 20, 110, 205, 524, 72
Monkey 2:
Monkey 3:

After round 6, the monkeys are holding items with these worry levels:
Monkey 0: 8, 70, 176, 26, 34
Monkey 1: 481, 32, 36, 186, 2190
Monkey 2:
Monkey 3:

After round 7, the monkeys are holding items with these worry levels:
Monkey 0: 162, 12, 14, 64, 732, 17
Monkey 1: 148, 372, 55, 72
Monkey 2:
Monkey 3:

After round 8, the monkeys are holding items with these worry levels:
Monkey 0: 51, 126, 20, 26, 136
Monkey 1: 343, 26, 30, 1546, 36
Monkey 2:
Monkey 3:

After round 9, the monkeys are holding items with these worry levels:
Monkey 0: 116, 10, 12, 517, 14
Monkey 1: 108, 267, 43, 55, 288
Monkey 2:
Monkey 3:

After round 10, the monkeys are holding items with these worry levels:
Monkey 0: 91, 16, 20, 98
Monkey 1: 481, 245, 22, 26, 1092, 30
Monkey 2:
Monkey 3:

...

After round 15, the monkeys are holding items with these worry levels:
Monkey 0: 83, 44, 8, 184, 9, 20, 26, 102
Monkey 1: 110, 36
Monkey 2:
Monkey 3:

...

After round 20, the monkeys are holding items with these worry levels:
Monkey 0: 10, 12, 14, 26, 34
Monkey 1: 245, 93, 53, 199, 115
Monkey 2:
Monkey 3:
</code></pre>

<p>Chasing all of the monkeys at once is impossible; you're going to have to focus on the <em>two most active</em> monkeys if you want any hope of getting your stuff back. Count the <em>total number of times each monkey inspects items</em> over 20 rounds:</p>
<pre><code><em>Monkey 0 inspected items 101 times.</em>
Monkey 1 inspected items 95 times.
Monkey 2 inspected items 7 times.
<em>Monkey 3 inspected items 105 times.</em>
</code></pre>
<p>In this example, the two most active monkeys inspected items 101 and 105 times. The level of <em>monkey business</em> in this situation can be found by multiplying these together: <code><em>10605</em></code>.</p>
<p>Figure out which monkeys to chase by counting how many items they inspect over 20 rounds. <em>What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans?</em></p>
</article>


In [12]:
from inspect import getsource
from math import prod
from pprint import pprint
from collections.abc import Callable


tests = [
    {
        "name": "Example",
        "s": """
            Monkey 0:
            Starting items: 79, 98
            Operation: new = old * 19
            Test: divisible by 23
                If true: throw to monkey 2
                If false: throw to monkey 3

            Monkey 1:
            Starting items: 54, 65, 75, 74
            Operation: new = old + 6
            Test: divisible by 19
                If true: throw to monkey 2
                If false: throw to monkey 0

            Monkey 2:
            Starting items: 79, 60, 97
            Operation: new = old * old
            Test: divisible by 13
                If true: throw to monkey 1
                If false: throw to monkey 3

            Monkey 3:
            Starting items: 74
            Operation: new = old + 3
            Test: divisible by 17
                If true: throw to monkey 0
                If false: throw to monkey 1
        """,
        "expected": 10605,
    },
]


class Monkey:

    def __init__(self, *args) -> None:
        name, items, operation, test, if_true, if_false = args

        self.name = name[:-1]
        self.items = [int(i) for i in items.removeprefix("Starting items:").split(",")]

        self.operation_exp = f"lambda old: {operation.split(" = ")[1]}"
        self.operation = eval(self.operation_exp)

        self.test = int(re.search(r"\d+", test).group())
        self.if_true = int(re.search(r"\d+", if_true).group())
        self.if_false = int(re.search(r"\d+", if_false).group())

        self.items_ispected = 0

    def inspect(self, monkeys: list[Monkey], prt: bool) -> None:
        if prt:
            print(f"{self.name}:")

        while self.items:
            item = self.items.pop(0)
            self.items_ispected += 1

            if prt:
                print(f"\tMonkey inspects an item with a worry level of {item}.")
            worry = self.operation(item)
            multiplyer = re.search(r"\d+", self.operation_exp)

            if prt:
                mult = multiplyer.group() if multiplyer is not None else "itself"
                print(f"\t\tWorry level is multiplied by {mult} to {worry}.")
            worry //= 3
            if prt:
                print(
                    f"\t\tMonkey gets bored with item. Worry level is divided by 3 to {worry}."
                )
            if worry % self.test == 0:
                if prt:
                    print(f"\t\tCurrent worry level is divisible by {self.test}.")
                    print(
                        f"\t\tItem with worry level {worry} is thrown to monkey {self.if_true}."
                    )

                monkeys[self.if_true].items.append(worry)
            else:
                if prt:
                    print(f"\t\tCurrent worry level is not divisible by {self.test}.")
                    print(
                        f"\t\tItem with worry level {worry} is thrown to monkey {self.if_false}."
                    )
                monkeys[self.if_false].items.append(worry)

    def __str__(self) -> str:
        return "\n".join(
            (
                "Monkey(",
                f"  name={self.name}, ",
                f"  items={self.items}, ",
                f"  operation={self.operation_exp}, ",
                f"  test={self.test_exp}",
                ")",
            )
        )

    __repr__ = __str__


class MonkeyInTheMiddle:
    def __init__(self, s: str) -> None:
        self.monkeys = [
            Monkey(*[c.strip() for c in m.splitlines()])
            for m in re.split(r"\n\s*\n", s.strip())
        ]

    def round(self, round: int, prt) -> None:
        for monkey in self.monkeys:
            monkey.inspect(self.monkeys, prt)

        if prt:
            print()
            print(
                f"After round {round}, the monkeys are holding items with these worry levels:"
            )
            print(
                "\n".join(
                    f"{m.name}: {', '.join(str(i) for i in m.items)}"
                    for m in self.monkeys
                )
            )
            print()

    def monkeybusiness(self, rounds=20, prt=False) -> int:
        for round in range(1, rounds + 1):
            self.round(round, prt)

        if prt:
            print()
            print(
                "\n".join(
                    f"{m.name} inspected items {m.items_ispected} times"
                    for m in self.monkeys
                )
            )
        return prod(sorted(m.items_ispected for m in self.monkeys)[-2:])


@test(tests=tests)
def test_part_I(s: str) -> int:
    return MonkeyInTheMiddle(s).monkeybusiness()


Test Example passed, for test_part_I.
Success


In [13]:
with open("../input/day11.txt") as f:
    puzzle = f.read()

print(f"Part 1: {MonkeyInTheMiddle(puzzle).monkeybusiness()}")

Part 1: 62491


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>62491</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You're worried you might not ever get your items back. So worried, in fact, that your relief that a monkey's inspection didn't damage an item <em>no longer causes your worry level to be divided by three</em>.</p>
<p>Unfortunately, that relief was all that was keeping your worry levels from reaching <em>ridiculous levels</em>. You'll need to <em>find another way to keep your worry levels manageable</em>.</p>
<p>At this rate, you might be putting up with these monkeys for a <em>very long time</em> - possibly <em><code>10000</code> rounds</em>!</p>
<p>With these new rules, you can still figure out the <span title="Monkey business monkey business monkey business, monkey numbers... is this working?">monkey business</span> after 10000 rounds. Using the same example above:</p>
<pre><code>== After round 1 ==
Monkey 0 inspected items 2 times.
Monkey 1 inspected items 4 times.
Monkey 2 inspected items 3 times.
Monkey 3 inspected items 6 times.

== After round 20 ==
Monkey 0 inspected items 99 times.
Monkey 1 inspected items 97 times.
Monkey 2 inspected items 8 times.
Monkey 3 inspected items 103 times.

== After round 1000 ==
Monkey 0 inspected items 5204 times.
Monkey 1 inspected items 4792 times.
Monkey 2 inspected items 199 times.
Monkey 3 inspected items 5192 times.

== After round 2000 ==
Monkey 0 inspected items 10419 times.
Monkey 1 inspected items 9577 times.
Monkey 2 inspected items 392 times.
Monkey 3 inspected items 10391 times.

== After round 3000 ==
Monkey 0 inspected items 15638 times.
Monkey 1 inspected items 14358 times.
Monkey 2 inspected items 587 times.
Monkey 3 inspected items 15593 times.

== After round 4000 ==
Monkey 0 inspected items 20858 times.
Monkey 1 inspected items 19138 times.
Monkey 2 inspected items 780 times.
Monkey 3 inspected items 20797 times.

== After round 5000 ==
Monkey 0 inspected items 26075 times.
Monkey 1 inspected items 23921 times.
Monkey 2 inspected items 974 times.
Monkey 3 inspected items 26000 times.

== After round 6000 ==
Monkey 0 inspected items 31294 times.
Monkey 1 inspected items 28702 times.
Monkey 2 inspected items 1165 times.
Monkey 3 inspected items 31204 times.

== After round 7000 ==
Monkey 0 inspected items 36508 times.
Monkey 1 inspected items 33488 times.
Monkey 2 inspected items 1360 times.
Monkey 3 inspected items 36400 times.

== After round 8000 ==
Monkey 0 inspected items 41728 times.
Monkey 1 inspected items 38268 times.
Monkey 2 inspected items 1553 times.
Monkey 3 inspected items 41606 times.

== After round 9000 ==
Monkey 0 inspected items 46945 times.
Monkey 1 inspected items 43051 times.
Monkey 2 inspected items 1746 times.
Monkey 3 inspected items 46807 times.

== After round 10000 ==
<em>Monkey 0 inspected items 52166 times.</em>
Monkey 1 inspected items 47830 times.
Monkey 2 inspected items 1938 times.
<em>Monkey 3 inspected items 52013 times.</em>
</code></pre>

<p>After 10000 rounds, the two most active monkeys inspected items 52166 and 52013 times. Multiplying these together, the level of <em>monkey business</em> in this situation is now <code><em>2713310158</em></code>.</p>
<p>Worry levels are no longer divided by three after each item is inspected; you'll need to find another way to keep your worry levels manageable. Starting again from the initial state in your puzzle input, <em>what is the level of monkey business after 10000 rounds?</em></p>
</article>


In [14]:
from inspect import getsource
from math import prod
from pprint import pprint
from collections.abc import Callable


tests = [
    {
        "name": "Example",
        "s": """
            Monkey 0:
            Starting items: 79, 98
            Operation: new = old * 19
            Test: divisible by 23
                If true: throw to monkey 2
                If false: throw to monkey 3

            Monkey 1:
            Starting items: 54, 65, 75, 74
            Operation: new = old + 6
            Test: divisible by 19
                If true: throw to monkey 2
                If false: throw to monkey 0

            Monkey 2:
            Starting items: 79, 60, 97
            Operation: new = old * old
            Test: divisible by 13
                If true: throw to monkey 1
                If false: throw to monkey 3

            Monkey 3:
            Starting items: 74
            Operation: new = old + 3
            Test: divisible by 17
                If true: throw to monkey 0
                If false: throw to monkey 1
        """,
        "expected": 2713310158,
    },
]


class MonkeyII(Monkey):

    def inspect(self, monkeys: list[Monkey], prt: bool) -> None:
        if prt:
            print(f"{self.name}:")

        while self.items:
            item = self.items.pop(0)
            item %= self.mod

            self.items_ispected += 1

            if prt:
                print(f"\tMonkey inspects an item with a worry level of {item}.")
            worry = self.operation(item)
            multiplyer = re.search(r"\d+", self.operation_exp)

            if prt:
                mult = multiplyer.group() if multiplyer is not None else "itself"
                print(f"\t\tWorry level is multiplied by {mult} to {worry}.")

            if worry % self.test == 0:
                if prt:
                    print(f"\t\tCurrent worry level is divisible by {self.test}.")
                    print(
                        f"\t\tItem with worry level {worry} is thrown to monkey {self.if_true}."
                    )

                monkeys[self.if_true].items.append(worry)
            else:
                if prt:
                    print(f"\t\tCurrent worry level is not divisible by {self.test}.")
                    print(
                        f"\t\tItem with worry level {worry} is thrown to monkey {self.if_false}."
                    )
                monkeys[self.if_false].items.append(worry)


class MonkeyInTheMiddleII(MonkeyInTheMiddle):
    def __init__(self, s: str) -> None:
        self.monkeys = [
            MonkeyII(*[c.strip() for c in m.splitlines()])
            for m in re.split(r"\n\s*\n", s.strip())
        ]

        # take the moulo (=prouctall monkey divisors) for worry level
        # this doesn't change the counts of the actions,
        # because it doesn't change the divisiablilty
        # But reduces the amount of work because ofu nhinged integer number sizes
        self.modulo = prod(m.test for m in self.monkeys)

        for monkey in self.monkeys:
            monkey.mod = self.modulo


@test(tests=tests)
def test_part_II(s: str) -> int:
    return MonkeyInTheMiddleII(s).monkeybusiness(rounds=10_000)


Test Example passed, for test_part_II.
Success


<link href="style.css" rel="stylesheet"></link>


In [15]:
print(f"Part 2: { MonkeyInTheMiddleII(puzzle).monkeybusiness(rounds=10_000)}")

Part 2: 17408399184


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>17408399184</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
